In [ ]:
import artm

print artm.version()

Создаю батчи

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='',
                                        data_format='bow_uci',
                                        collection_name='wikistem',
                                        target_folder='wikistem_batches',                                 
                                        batch_size=1000)

batch_vectorizer = artm.BatchVectorizer(data_path='',
                                        data_format='bow_uci',
                                        collection_name='wikilemm',
                                        target_folder='wikilemm_batches',
                                        batch_size=1000)

Создаю на основе батчей словари и сохраняю их

In [ ]:
dictionary_stem = artm.Dictionary()
dictionary_stem.gather(data_path='wikistem_batches')
dictionary_stem.save_text(dictionary_path='wikistem_batches/dictionary_stem.txt')

dictionary_lemm = artm.Dictionary()
dictionary_lemm.gather(data_path='wikilemm_batches')
dictionary_lemm.save_text(dictionary_path='wikilemm_batches/dictionary_lemm.txt')

Подгрузка уже существующих батчей и словарей

In [ ]:
batch_vectorizer_stemm = artm.BatchVectorizer(data_path='wikistem_batches',
                                        data_format='batches')

dictionary_stem = artm.Dictionary()
dictionary_stem.load_text(dictionary_path='wikistem_batches/dictionary_stem.txt')

batch_vectorizer_lemm = artm.BatchVectorizer(data_path='wikilemm_batches',
                                        data_format='batches')

dictionary_lemm = artm.Dictionary()
dictionary_lemm.load_text(dictionary_path='wikilemm_batches/dictionary_lemm.txt')

## Оценивание LDA

Теперь создадим саму тематическую модель, указав число тем и параметр числа прохода по документу, гиперпараметры сглаживания матриц Φ и Θ, а также используемый словарь. Кроме того, попросим сохранять матрицу Θ, чтобы потом на неё можно было взглянуть.

Здесь же можно указать параметр num_processors, отвечающий за число потоков, которые будут использованы для вычислений на машине.

In [ ]:
lda = artm.LDA(num_topics=400, alpha=0.01, beta=0.001,
               num_document_passes=10, dictionary=dictionary_lemm,
               cache_theta=True,num_processors=8)


In [ ]:
lda.fit_offline(batch_vectorizer=batch_vectorizer_lemm, num_collection_passes=20)

Можно посмотреть на значения перпелексии на каждой итерации прохода по коллекции:

In [ ]:
lda.perplexity_value

Наконец, можно посмотреть на наиболее вероятные слова в каждой теме. Они выдаются в виде списка списков строк (каждый внутренний список соответствуюет одной теме, по порядку). Выведем их, предварительно отформатировав:

In [ ]:
top_tokens = lda.get_top_tokens(num_tokens=30)
for i, token_list in enumerate(top_tokens):
    print 'Topic #{0}: {1}'.format(i, token_list)

Можно получить доступ к матрицам с помощью следующих вызовов:

In [ ]:
phi = lda.phi_
theta = lda.get_theta()

Опишем ещё ряд возможностей модели artm.LDA.

Во-первых - это построение матрицы Θ для новых документов при наличии обученной модели:

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='kos_batches_test')
theta_test = lda.transform(batch_vectorizer=test_batch_vectorizer)

Во-вторых, в том случае, если требуется производить различную регуляризацию каждой из тем в матрице ΦΦ, вместо скалярного значения beta можно задать список гиперпараметров, длиной в число тем, и каждая тема будет регуляризована со своим гиперпараметром:

In [ ]:
beta = [0.1] * num_topics  # change as you need
lda = artm.LDA(num_topics=15, alpha=0.01, beta=beta, num_document_passes=5, dictionary=dictionary, cache_theta=True)

Сохраняем нужную нам информацию. Вектор из пералексий, самые популярные слова и саму модель.

In [ ]:
import pickle

# Сохраняем вектор из перплексий
a=lda.perplexity_value
with open('lda_lemm_400_perp.pickle', 'wb') as handle:
    pickle.dump(a, handle)

# Сохраняем топики из слов
top_tokens = lda.get_top_tokens(num_tokens=30)
with open('lda_lemm_400_top_tokens.pickle', 'wb') as handle:
    pickle.dump(top_tokens, handle)    
    
# Сохраняем модель
lda.save("lda_400_lemm")


## PLSA обычная и с небольшой частичкой волшебства

In [ ]:
batch_vectorizer_lemm = artm.BatchVectorizer(data_path='wikilemm_batches',
                                        data_format='batches')

dictionary_lemm = artm.Dictionary()
dictionary_lemm.load_text(dictionary_path='wikilemm_batches/dictionary_lemm.txt')

T = 400
my_dictionary = dictionary_lemm
batch_vectorizer = batch_vectorizer_lemm

# создаём модель
model = artm.ARTM(num_topics=T, dictionary=my_dictionary, cache_theta=False)
# Когда в коллекции много документов, матрицу лучше не запоминать!

# вешаем на неё перплексию
model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                      dictionary=my_dictionary))

# Вешаем метрики разряженности матриц
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))

# Вешаем запоминатель самых вероятных слов, будем запоминать 30 штук
model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=30))


* Оффлайновый алгоритм: много проходов по коллекции, один проход по документу (опционально), обновление Φ в конце каждого прохода. Используйте, если у Вас маленькая коллекция.

* Онлайновый алгоритм: один проход по коллекции (опционально), много проходов по документу, обновление Φ раз в заданное количество батчей. Используйте при большой коллекции, и коллекции с быстро меняющейся тематикой.

In [ ]:
model.num_document_passes = 10  # сколько делать проходов по каждому документу

import time
start_time = time.time() # время работы кода 
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=20)
print "--- %s seconds ---" % (time.time() - start_time)  #время работы кода
#Юпи!


In [ ]:
# распечатаем все значения перплесии и других метрик
print model.score_tracker['perplexity_score'].value      # .last_value
print model.score_tracker['sparsity_phi_score'].value    # .last_value
print model.score_tracker['sparsity_theta_score'].value  # .last_value

In [ ]:
# Ежели нас что-то не устраивает, то дообучаем модель делая еще например 15 итераций.
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

Да, последний момент: метрики выгружаются из ядра при каждом обращении, поэтому для таких больших метрик, как топ-слова (или ядровые характеристики, о которых можно прочитать по данным выше ссылкам), лучше завести переменную, в которую Вы один раз всё выгрузите, а потом уже работать с ней. Поехали, просмотрим топ-слова последовательно в цикле по именам тем модели:

In [ ]:
saved_top_tokens = model.score_tracker['top_tokens_score'].last_tokens

for topic_name in model.topic_names:
    print saved_top_tokens[topic_name]

Наверняка темы получились не слишком впечатляющими, да? И вот сейчас на сцену выходят они, озарённые сиянием прожекторов звёзды Тематической империи - регуляризаторы! Их задача - помочь Вам сделать модель как можно более хорошей и милой.

Списки регуляризаторов и их параметров можно посмотреть здесь http://bigartm.readthedocs.org/en/master/python_interface/regularizers.html. Код работы с регуляризаторами очень похож на код работы с метриками (они всё-таки дальние родственники). Давайте в нашу модель добавим три регуляризатора: разреживание Φ, разреживание Θ и декорреляция тем. Последний в поте лица старается сделать темы как можно более различными.


In [ ]:
model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='sparse_phi_regularizer'))
model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='sparse_theta_regularizer'))
model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer'))

Возможно, у Вас вызывает когнитивный диссонанс регуляризатор SmoothSparsePhi\Theta, это он что же, и сглаживает, и разреживает? И мой ответ - именно так. Только не "и", а "или" (хотя для ΦΦ можно и "и", об этом дальше будет). Он может и то, и то, его действия будут зависеть от того, каким Вы зададите его коэффициент регуляризации ττ (если не знаете, что это - обращайтесь к работам архимага). ττ > 0 - будет сглаживать, ττ < 0 - разреживать. По-умолчанию все регуляризаторы получают ττ = 1.0, что, как правило, совершенно не подходит. Выбор подходящего ττ - эвристика, искусство заклинателя, порой приходится провести десятки опытов, чтобы подобрать хорошие значений. Я не буду здесь этим заниматься, просто покажу, как выставлять эти значения:

In [ ]:
model.regularizers['sparse_phi_regularizer'].tau = -5*1e5
model.regularizers['sparse_theta_regularizer'].tau = -1000
model.regularizers['decorrelator_phi_regularizer'].tau = 1e+5

# В темах много общеупотребительных слов (так называемой, фоновой лексики). 
# Чтобы этого избежать, будем использовать разреживающий регуляризатор для матрицы фи.
# Он будет подавлять слова, которые имеют большую частоту во всей коллекции.

# Меняем так, чтобы добиться разряжености - более высокие значения метрик (частота нулей в матрице) 
# и интерпретируемости тем. Лучше всего применять его после того как модель сошлась чтобы добиться интерпретации!
# То есть сначала обучили модель, после ввели метрики, после дообучили

model.regularizers['sparse_phi_regularizer'].tau = -100
model.regularizers['sparse_phi_regularizer'].tau = -5*1e4 

# если вы хотите применять регуляризатор только к некоторым модальностям,
# указывайте это в параметре class_ids: class_ids=["text"]

Выставленные значения стандартны, но при неблагоприятном стечении обстоятельств могут либо не оказать на модель существенного влияния, либо заставить её корчиться в муках. Берегите свои модели, подбирайте коэффициенты тщательно.

In [ ]:
# дообучаем модель
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=5)

Дальше можно снова смотреть на метрики, поправлять коэффиценты ττ регуляризаторов и т.д.

Забыл упомянуть, что регуляризаторам, как и метрикам, тоже можно объяснить, какие темы можно трогать, а какие - нельзя (они всё-таки неглупые ребята). Делается это в полной аналогии с тем, как мы проделали это для метрик. Не буду приводить пример из принципа, сами догадайтесь, это элементарно. (http://nbviewer.jupyter.org/github/bigartm/bigartm-book/blob/master/ARTM_tutorial_Fun.ipynb)

In [ ]:
import pickle
# Сохраняем топики из слов
top_tokens = model.score_tracker['top_tokens_score'].last_tokens
with open('plsa_lemm_200_top_tokens.pickle', 'wb') as handle:
    pickle.dump(top_tokens, handle)   

# Сохраняем метрики
a1 = model.score_tracker['perplexity_score'].value      # .last_value
a2 = model.score_tracker['sparsity_phi_score'].value    # .last_value
a3 = model.score_tracker['sparsity_theta_score'].value  # .last_value

a = {'perep':a1,'phi':a2,'theta':a3}
with open('plsa_lemm_200_scores.pickle', 'wb') as handle:
    pickle.dump(a, handle)